<a href="https://colab.research.google.com/github/ashmithareddy20/T247_Hackathon/blob/main/T247_Hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import random
import subprocess
import time

SUMATRA_PATH = "/content/seed.pdf"
OUTPUT_DIR = "fuzz_output"
CRASH_DIR = os.path.join(OUTPUT_DIR, "crashes")
NUM_TEST_CASES = 100
SEED_FILE = "seed.pdf"

if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)
if not os.path.exists(CRASH_DIR):
    os.makedirs(CRASH_DIR)

def generate_mutated_pdf(seed_file, output_file):
    """
    Generates a simple mutated PDF by randomly altering a few bytes.
    This is a very basic mutation strategy and not representative of
    sophisticated fuzzers.
    """
    try:
        with open(seed_file, "rb") as f:
            pdf_content = bytearray(f.read())
    except FileNotFoundError:
        print(f"Error: Seed file '{seed_file}' not found.")
        return None

    num_mutations = random.randint(1, 5)
    for _ in range(num_mutations):
        if pdf_content:
            index_to_mutate = random.randrange(len(pdf_content))
            random_byte = random.randint(0, 255)
            pdf_content[index_to_mutate] = random_byte

    try:
        with open(output_file, "wb") as f:
            f.write(pdf_content)
        return output_file
    except Exception as e:
        print(f"Error writing mutated file: {e}")
        return None

def run_sumatra(pdf_file):
    """
    Runs Sumatra PDF with the given file and returns the exit code.
    """
    try:
        process = subprocess.Popen([SUMATRA_PATH, pdf_file],
                                   stdout=subprocess.PIPE,
                                   stderr=subprocess.PIPE)
        stdout, stderr = process.communicate(timeout=5)
        return process.returncode, stdout.decode(errors='ignore'), stderr.decode(errors='ignore')
    except subprocess.TimeoutExpired:
        process.kill()
        return -99, "", "Timeout"
    except FileNotFoundError:
        print(f"Error: Sumatra PDF not found at '{SUMATRA_PATH}'. Update SUMATRA_PATH.")
        return -1, "", "Sumatra not found"
    except Exception as e:
        return -2, "", f"Error running Sumatra: {e}"

def analyze_crash(return_code, stdout, stderr, input_file):
    """
    A very basic crash analysis. Looks for non-zero exit codes.
    More sophisticated analysis would involve examining error messages,
    memory dumps, etc.
    """
    if return_code != 0 and return_code not in [3221225477]:
        print(f"\nPotential crash detected with file: {input_file}")
        print(f"Return Code: {return_code}")
        print("Stdout:")
        print(stdout)
        print("Stderr:")
        print(stderr)
        crash_filename = os.path.join(CRASH_DIR, os.path.basename(input_file))
        os.rename(input_file, crash_filename)
        print(f"Crash input saved to: {crash_filename}")
        return True
    return False


if __name__ == "__main__":
    print("Simple Python-based conceptual fuzzer (not a full harness)")
    if not os.path.exists(SEED_FILE):
        print(f"Error: Seed file '{SEED_FILE}' is required. Create a sample PDF.")
    elif not os.path.exists(SUMATRA_PATH):
        print(f"Error: Sumatra PDF not found at '{SUMATRA_PATH}'. Please update the path.")
    else:
        for i in range(NUM_TEST_CASES):
            mutated_filename = os.path.join(OUTPUT_DIR, f"mutated_{i}.pdf")
            mutated_file = generate_mutated_pdf(SEED_FILE, mutated_filename)

            if mutated_file:
                print(f"Testing: {mutated_file}")
                return_code, stdout, stderr = run_sumatra(mutated_file)
                analyze_crash(return_code, stdout, stderr, mutated_file)
                time.sleep(0.1)

        print("\nFuzzing finished.")
        print(f"Check '{CRASH_DIR}' for potential crashes.")

Simple Python-based conceptual fuzzer (not a full harness)
Testing: fuzz_output/mutated_0.pdf

Potential crash detected with file: fuzz_output/mutated_0.pdf
Return Code: -2
Stdout:

Stderr:
Error running Sumatra: [Errno 13] Permission denied: '/content/seed.pdf'
Crash input saved to: fuzz_output/crashes/mutated_0.pdf
Testing: fuzz_output/mutated_1.pdf

Potential crash detected with file: fuzz_output/mutated_1.pdf
Return Code: -2
Stdout:

Stderr:
Error running Sumatra: [Errno 13] Permission denied: '/content/seed.pdf'
Crash input saved to: fuzz_output/crashes/mutated_1.pdf
Testing: fuzz_output/mutated_2.pdf

Potential crash detected with file: fuzz_output/mutated_2.pdf
Return Code: -2
Stdout:

Stderr:
Error running Sumatra: [Errno 13] Permission denied: '/content/seed.pdf'
Crash input saved to: fuzz_output/crashes/mutated_2.pdf
Testing: fuzz_output/mutated_3.pdf

Potential crash detected with file: fuzz_output/mutated_3.pdf
Return Code: -2
Stdout:

Stderr:
Error running Sumatra: [Errno 